In [1]:
!pip install unidecode

# Libraries

In [0]:
import pandas as pd

import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter
import os
from argparse import Namespace

import unidecode
import random

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder

# Data preprocessing

In [0]:
reviews = pd.read_table('data/reviews.tsv')

In [0]:
reviews.dropna(subset=['content'], inplace=True)

In [0]:
def get_data_from_dataframe(df, batch_size, seq_size):
    
    text = " ".join(df.content.apply(unidecode.unidecode).values.flatten())
    
    text = text.split()

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text

In [0]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

## KNN

In [0]:
def column_label_encoding(df, le_colname):
  le = LabelEncoder()
  df[le_colname] = le.fit_transform(df[le_colname].to_list()) 
  return df

def mean_score_encoding(df, grouping_columns, target_columns):
  for target_column in target_columns:
    mean_group = df.groupby(grouping_columns)[target_column].mean().reset_index()
    mean_group.columns = grouping_columns + ['_'.join(grouping_columns) + '_mean_' + target_column]
    df = df.merge(mean_group, on=grouping_columns)
  return df

def convert_string_to_date(date_time_str):
  conversion = datetime.datetime.strptime(date_time_str, "%Y-%m-%d")
  return conversion

def change_date_for_column(df, column):
    return df[column].apply(convert_string_to_date)

def preprocess_knn(df):

  df = df[["artist", 
           "score", 
           "pub_date", 
           "best_new_music", 
           "genre", 
           "label", 
           "acousticness",
           "danceability",
           "energy",
           "instrumental",
           "liveness",
           "loudness",
           "speechiness",
           "tempo",
           "valence",
           "popularity"
  ]]
  df = column_label_encoding(df, 'label')
  df = mean_score_encoding(df, ['artist'], ['score'])
  df['pub_date'] = change_date_for_column(df, 'pub_date')
  df['pub_date'] = pd.to_numeric(df['pub_date'], errors='coerce')
  df = pd.get_dummies(df.drop(["artist"], axis=1))

  return df

In [0]:
def perform_KNN(df, n_neighbors):
    neighs = NearestNeighbors(n_neighbors=n_neighbors)
    neigs = neighs.fit(df)
    _, indices = neighs.kneighbors(reviews_knn)
        
    return indices

In [9]:
# Get nearest neighbors for each album
reviews_knn = preprocess_knn(reviews)
neighborhoods = perform_KNN(reviews_knn, 16)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# Selecting a random album to generate a review
random_review_index = random.randint(0, reviews.shape[0])
test_album = reviews.iloc[random_review_index]
reviews = reviews.drop([random_review_index])

In [0]:
# Selecting reviews from nearest neighbors
reviews_from_cluster = reviews.iloc[neighborhoods[random_review_index]]

## RNN

In [0]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state
        
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

In [0]:
def get_loss_and_train_op(net, lr=0.001):
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)

        return criterion, optimizer

In [0]:
flags = Namespace(
    seq_size=32,
    batch_size=32,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['This', 'album'],
    predict_top_k=5,
    checkpoint_path='checkpoint',
)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [16]:
reviews_index_to_train = reviews_from_cluster.index.tolist() + random.sample(range(reviews.shape[0]), 256)

int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_dataframe(
    reviews.iloc[reviews_index_to_train], 
    flags.batch_size, 
    flags.seq_size
)

net = RNNModule(n_vocab, flags.seq_size, flags.embedding_size, flags.lstm_size)
net = net.to(device)

criterion, optimizer = get_loss_and_train_op(net, 0.05)

iteration = 0

Vocabulary size 36315


# Training

In [0]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

In [18]:
n_epochs = 10
for e in range(n_epochs):
    batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
    state_h, state_c = net.zero_state(flags.batch_size)

    # Transfer data to GPU
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for x, y in batches:
        iteration += 1

        # Tell it we are in training mode
        net.train()

        # Reset all gradients
        optimizer.zero_grad()

        # Transfer data to GPU
        x = torch.tensor(x).to(device)
        y = torch.tensor(y).to(device)

        logits, (state_h, state_c) = net(x, (state_h, state_c))
        loss = criterion(logits.transpose(1, 2), y)

        state_h = state_h.detach()
        state_c = state_c.detach()

        loss_value = loss.item()

        # Perform back-propagation
        loss.backward()
        
        _ = torch.nn.utils.clip_grad_norm_(net.parameters(), flags.gradients_norm)

        # Update the network's parameters
        optimizer.step()

        if iteration % 100 == 0:
            print('Epoch: {}/{}'.format(e, n_epochs),
                  'Iteration: {}'.format(iteration),
                  'Loss: {}'.format(loss_value))

    predict(device, net, flags.initial_words, n_vocab,
                    vocab_to_int, int_to_vocab, top_k=5)

Epoch: 0/10 Iteration: 100 Loss: 8.423583984375
This album that and in its own to the music of a bit that the music of an of its music to his music to a and the album is a few and and the first to be a few of an artist in a more and a bit to a and of the first to the album is the album is his of his first in his own to be a more in the same and is a little in its same of a little and and of his own the album the first to an to be and the first time
Epoch: 1/10 Iteration: 200 Loss: 7.23267936706543
Epoch: 1/10 Iteration: 300 Loss: 7.3150129318237305
This album that and in its own to the music of a bit that the music of an of its music to his music to a and the album is a few and and the first to be a few of an artist in a more and a bit to a and of the first to the album is the album is his of his first in his own to be a more in the same and is a little in its same of a little and and of his own the album the first to an to be and the first time of the album's track of his own jazz to 

## Retrain

In [0]:
seq_size = 32
batch_size = 8

text = " ".join(reviews.iloc[reviews_from_cluster.index].content.apply(unidecode.unidecode).values.flatten())
    
text = text.split()
 
int_text = [vocab_to_int[w] for w in text]
num_batches = int(len(int_text) / (seq_size * batch_size))
in_text = int_text[:num_batches * batch_size * seq_size]
out_text = np.zeros_like(in_text)
out_text[:-1] = in_text[1:]
out_text[-1] = in_text[0]
in_text = np.reshape(in_text, (batch_size, -1))
out_text = np.reshape(out_text, (batch_size, -1))

In [21]:
n_epochs = 3
for e in range(n_epochs):
    batches = get_batches(in_text, out_text, batch_size, seq_size)
    state_h, state_c = net.zero_state(batch_size)

    # Transfer data to GPU
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for x, y in batches:
        iteration += 1

        # Tell it we are in training mode
        net.train()

        # Reset all gradients
        optimizer.zero_grad()

        # Transfer data to GPU
        x = torch.tensor(x).to(device)
        y = torch.tensor(y).to(device)

        logits, (state_h, state_c) = net(x, (state_h, state_c))
        loss = criterion(logits.transpose(1, 2), y)

        state_h = state_h.detach()
        state_c = state_c.detach()

        loss_value = loss.item()

        # Perform back-propagation
        loss.backward()
        
        _ = torch.nn.utils.clip_grad_norm_(net.parameters(), flags.gradients_norm)

        # Update the network's parameters
        optimizer.step()

        if iteration % 100 == 0:
            print('Epoch: {}/{}'.format(e, n_epochs),
                  'Iteration: {}'.format(iteration),
                  'Loss: {}'.format(loss_value))

    predict(device, net, flags.initial_words, n_vocab,
                    vocab_to_int, int_to_vocab, top_k=5)

This album that and in its own to the music of a bit that the music of an of its music to his music to a and the album is a few and and the first to be a few of an artist in a more and a bit to a and of the first to the album is the album is his of his first in his own to be a more in the same and is a little in its same of a little and and of his own the album the first to an to be and the first time of the album's track of his own jazz to an attempt in a little more of the first couple of his music on its most innovative on the other own and and as an artist the most part, and is the most the music that is the music and the music and is that he sings, as an artist and the album's of his own own and a little bit and and as well for a few of an entire of its first house, a little more is as well for his and is an artist and is that he is an album is his banjo of a bit and the music is that it seems to make you know, he is the first time the music on his beat in their most in an obsolet